# Data Cleaning and Preprocessing 

#### adding required packages

In [18]:
import time
import pandas as pd
import numpy as np
import xmltodict

#### specifying the saving location 

In [59]:
savingLoc = "Y:/ZahraEftekhar/phase4/"

## 1) keeping allowed users from the MATSim files

In this step we use the `1.experienced_plans.xml` and `snapShot.CSV` file in the MATsim output folder. We would like to only keep the users with the `car` mode. Furthermore, we remove users with zero duration activities. The users we keep are the same in both of the mentioned files. Finally, after this data cleaning we end up with about 21,000 users. 

#### reading `1.experienced_plans.xml` and converting it to a dataframe:

In [24]:
#we time the process
startTime = time.time()

# reading the MATSim output into a dict file
tree = xmltodict.parse(open("{a}1.experienced_plans.xml".format(a=savingLoc),"rb"))
# tree = xmltodict.parse(open("/data/zahraeftekhar/research_temporal/input_base/1.experienced_plans.xml","rb"))

# root is a list of plans of the users 
root = tree['population']['person'] 
del tree
print(time.time() - startTime,'seconds')

263.1187858581543 seconds


#### keeping only users with the `car` mode and removing users with unacceptable travel diary:

Here, we want to extract the user IDs that need to be removed them from our data set. This includes the users with non-positive activity durations, users with less than three activities which show no travel, etc. Also we only consider users with the mode `car`. 

In [26]:
#we time the process
startTime = time.time()

RemoveIDs = []
for child in root:
    try:
        # remove IDs that did not travel
        if len(child["plan"]["activity"])<3:
            RemoveIDs += [child['@id']]

        # remove IDs that used any mode other than `car`
        elif not all(flag["@mode"] == "car" for flag in child['plan']['leg']):
            RemoveIDs += [child['@id']]

        # remove IDs that their 1st and last activity are not similar
        # this is done to be able to have a round travel diary
        elif child["plan"]["activity"][0]["@type"]!=child["plan"]["activity"][-1]["@type"]:
            RemoveIDs += [child['@id']]

        # remove IDs with zero duration activities:
        # 1) removing `generic`legs because they usually lead to zero duration activities
        elif not all(flag['route']['@type'] != 'generic' for flag in child['plan']['leg']):
            RemoveIDs += [child['@id']]

        # 2) remove the rest of IDs with zero or negative duration activities
        elif not all((pd.to_timedelta(flag['@end_time'])).total_seconds() -
                     (pd.to_timedelta(flag['@start_time'])).total_seconds() > 0
                     for flag in child['plan']['activity'][1:-1]):
            RemoveIDs += [child['@id']]
    except KeyError:
        RemoveIDs += [child['@id']]
print(time.time() - startTime,'seconds')

3.3839805126190186 seconds


### 1.1) removing `RemoveIDs` from travel diaries (`1.experienced_plans.xml`)

Here, we remove the IDs inside `RemoveIDs` from the travel diaries dataset.

In [191]:
# number of users after data cleaning
print(len(root)-len(RemoveIDs))

cleanedData = pd.DataFrame.from_dict(root)
del root
cleanedData.columns = ['id', 'plan']
cleanedData.set_index('id',inplace=True) 
cleanedData.drop(RemoveIDs, inplace=True)
cleanedData.head()

22208


,plan
id,
1,"{'@score': '103.65231197449688', '@selected': ..."
10,"{'@score': '101.84174881258225', '@selected': ..."
100007,"{'@score': '105.56505677006494', '@selected': ..."
100009,"{'@score': '104.99507686427171', '@selected': ..."
10001,"{'@score': '105.92977435468114', '@selected': ..."


Now, we have to fix the error in the location coordinates of the travel diaries (for some unknown reasons). The location problem only exist in the travel diaries NOT the snapShot file (representing the GSM data). At the end we generate a `dict` of final round travel diaries with `key` of each vehicle ID and their associated panda DataFrame with columns: `VEHICLE`,`activityType`,`x`,`y`,`start`,`end`.

In [193]:
links = (xmltodict.parse(open("{a}output_network.xml".format(a=savingLoc),"rb")))["network"]["links"]["link"]
nodes = (xmltodict.parse(open("{a}output_network.xml".format(a=savingLoc),"rb")))["network"]["nodes"]["node"]

In [195]:
#we time the process
startTime = time.time()

trueLocations = {}
for ID in cleanedData.index:
    legList = [(flag['route']) for flag in cleanedData.loc[ID,:]['plan']['leg']]
    activityList = cleanedData.loc[ID,:]['plan']['activity']
    person = pd.DataFrame(columns=['VEHICLE','activityType','x','y','start','end'])
    i=0
    for i in range(len(legList)):
        trueloc = pd.DataFrame(columns=['VEHICLE','activityType','x','y','start','end'])
        trueloc.loc[0,'VEHICLE'] = ID
        trueloc.loc[0,'activityType'] = activityList[i+1]['@type']
       
        trueloc.loc[0,'start'] = activityList[i+1]['@start_time']
        try:
            trueloc.loc[0,'end'] = activityList[i+1]['@end_time']
        except KeyError:
            trueloc.loc[0,'end'] = activityList[0]['@end_time']
        linkID = legList[i]['@end_link']
        for j in range(len(links)):
            if links[j]["@id"]==linkID:
                nod = links[j]["@to"]
                for k in range(len(nodes)):
                    if nodes[k]['@id']==nod:
                        trueloc.loc[0,'x'] = nodes[k]['@x']
                        trueloc.loc[0,'y'] = nodes[k]['@y']
        person = person.append(trueloc)
    trueLocations[ID] = person
import pickle
with open('{a}1.trueLocExperienced.pickle'.format(a=savingLoc),'wb') as handle:
    pickle.dump(trueLocations, handle, protocol=pickle.HIGHEST_PROTOCOL)
print((time.time() - startTime)//60,'minutess')

24.0 minutess


### 1.2) removing `RemoveIDs` from GSm data (`snapShot.CSV`)

So far we cleaned the `1.experienced_plans.xml` file with the right users, location coordinates and  and travel diaries. Finally, we saved it as a `dict` file named `1.trueLocExperienced.pickle`.